# 获取河流冲积物区域

In [14]:
import os
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.features import geometry_mask
from scipy.ndimage import generic_filter, distance_transform_edt
from scipy.interpolate import griddata

In [15]:

def calculate_slope(dem, cell_size):
    dy, dx = np.gradient(dem, cell_size)
    slope_radians = np.arctan(np.sqrt(dx*dx + dy*dy))
    return np.degrees(slope_radians)

def calculate_elevation_difference(dem, river_raster, cell_size, max_distance=1000):
    # 创建距离栅格
    distance = distance_transform_edt(~river_raster) * cell_size
    
    # 获取河流像元的坐标和高程
    river_coords = np.column_stack(np.where(river_raster))
    river_elevations = dem[river_raster]
    
    # 对每个像元，找到最近的河流像元并计算高程差
    y, x = np.mgrid[0:dem.shape[0], 0:dem.shape[1]]
    positions = np.column_stack((y.ravel(), x.ravel()))
    elevation_diff = griddata(river_coords, river_elevations, positions, method='nearest').reshape(dem.shape)
    elevation_diff = dem - elevation_diff
    
    # 将距离超过max_distance的区域设置为NaN
    elevation_diff[distance > max_distance] = np.nan
    
    return elevation_diff

def local_relief(dem, size=5):
    return generic_filter(dem, np.ptp, size=size)

def identify_alluvial_areas(river_vector_path, dem_path, save_path, buffer_distance=500, slope_threshold=10, elevation_threshold=8, relief_threshold=8):
    # 读取数据
    with rasterio.open(dem_path) as src:
        dem = src.read(1)
        transform = src.transform
        crs = src.crs
    
    rivers = gpd.read_file(river_vector_path)
    
    # 创建河流栅格
    river_raster = geometry_mask(rivers.geometry, out_shape=dem.shape, transform=transform, invert=True)
    
    # 计算像元大小
    cell_size = transform[0]
    
    # 计算坡度
    slope = calculate_slope(dem, cell_size)
    
    # 计算高程差
    elevation_diff = calculate_elevation_difference(dem, river_raster, cell_size, max_distance=buffer_distance)
    
    # 计算局部起伏
    relief = local_relief(dem)
    
    
    # 识别潜在的冲积区域
    potential_alluvium = (
        (slope < slope_threshold) &
        (elevation_diff < elevation_threshold) &
        (relief < relief_threshold) &
        (~np.isnan(elevation_diff))
    )
    
    # 保存结果
    with rasterio.open(os.path.join(save_path, 'potential_alluvium_500_10_8_8.tif'), 'w', driver='GTiff',
                       height=dem.shape[0], width=dem.shape[1],
                       count=1, dtype=rasterio.uint8,
                       crs=crs, transform=transform) as dst:
        dst.write(potential_alluvium.astype(rasterio.uint8), 1)
    
    print("分析完成，结果已保存为 'potential_alluvium_500_10_8_8.tif'")



In [16]:
save_path = r"C:\Users\Runker\Desktop\River"
# 使用函数
identify_alluvial_areas(r"C:\Users\Runker\Desktop\SBRIVER\river.shp", r"C:\Users\Runker\Desktop\River\dem.tif",save_path)


分析完成，结果已保存为 'potential_alluvium_500_10_8_8.tif'


In [30]:
import rasterio
import numpy as np
from rasterio import features
import geopandas as gpd
from shapely.geometry import shape, MultiPolygon, Polygon
from shapely.ops import unary_union
import pygeos

def enhanced_alluvial_processing(input_raster_path, output_vector_path, min_area_size=500, buffer_distance=100):
    # 读取栅格数据
    with rasterio.open(input_raster_path) as src:
        raster_data = src.read(1)
        transform = src.transform
        crs = src.crs
    
    # 确保我们只处理值为1的区域
    raster_data = (raster_data == 1).astype(np.uint8)
    
    # 将栅格转换为矢量
    shapes = features.shapes(raster_data, transform=transform)
    geometries = [shape(geom) for geom, val in shapes if val == 1]
    
    # 创建GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': geometries}, crs=crs)
    
    # 将多部件几何体转换为单部件
    gdf = gdf.explode(index_parts=True)
    
    # 使用pygeos进行高效的空间操作
    pygeos_geoms = gdf.geometry.apply(lambda geom: pygeos.from_shapely(geom))
    
    # 对每个部件单独应用缓冲区操作
    buffered = pygeos.buffer(pygeos_geoms, buffer_distance)
    reverse_buffered = pygeos.buffer(buffered, -buffer_distance)
    
    # 处理重叠问题
    union = pygeos.union_all(reverse_buffered)
    result = pygeos.get_parts(union)
    
    # 转回Shapely几何对象
    final_geoms = [pygeos.to_shapely(geom) for geom in result]
    
    # 创建最终的GeoDataFrame
    final_gdf = gpd.GeoDataFrame({'geometry': final_geoms}, crs=crs)
    
    # 计算面积并添加为新字段
    final_gdf['area'] = final_gdf.geometry.area
    
    # 基于面积阈值筛选
    final_gdf = final_gdf[final_gdf['area'] >= min_area_size]
    
    # 简化几何形状以减少复杂性
    final_gdf['geometry'] = final_gdf.geometry.simplify(tolerance=1, preserve_topology=True)
    
    # 拓扑检查和修复
    def check_and_fix_topology(geom):
        if not geom.is_valid:
            print(f"Invalid geometry found: {pygeos.get_reason_invalid(pygeos.from_shapely(geom))}")
            geom = geom.buffer(0)  # 尝试修复
            if not geom.is_valid:
                print(f"Unable to fix geometry")
                return None
        return geom
    
    final_gdf['geometry'] = final_gdf.geometry.apply(check_and_fix_topology)
    
    # 移除任何无效的几何形状
    final_gdf = final_gdf.dropna(subset=['geometry'])
    
    # 保存为矢量文件
    final_gdf.to_file(output_vector_path)
    
    print(f"处理完成，结果已保存为 '{output_vector_path}'")
    print(f"最终处理的多边形数量: {len(final_gdf)}")
    print(f"最小面积: {final_gdf['area'].min():.2f}, 最大面积: {final_gdf['area'].max():.2f}")



In [31]:
# 使用函数
input_raster = r"C:\Users\Runker\Desktop\River\potential_alluvium_500_10_8_8.tif"
output_vector = r"C:\Users\Runker\Desktop\River\enhanced_alluvial_areas.shp"
enhanced_alluvial_processing(input_raster, output_vector, min_area_size=500, buffer_distance=100)

处理完成，结果已保存为 'C:\Users\Runker\Desktop\River\enhanced_alluvial_areas.shp'
最终处理的多边形数量: 2937
最小面积: 500.19, 最大面积: 8389618.20
